## Colab Setup (Skip if run locally)

In [ ]:
# Clone the repository and move its contents into the top level folder of the colab instance
# (Uncomment the following lines)
#!git clone https://github.com/yozoon/CosRec.git
#!mv CosRec/* .

In [ ]:
# (optional) Upload the saved checkpoint file (*.pth.tar),
# if training should continue from that saved state (change the saved_state_path
# accordingly)
from google.colab import files
files.upload()

## Interactive Environment for CosRec Model Training and Evaluation

In [ ]:
from dataclasses import dataclass, field
from typing import List, Tuple

import numpy as np
import torch
from IPython.display import Markdown, display

from evaluation import evaluate_ranking
from interactions import Interactions
from train import Recommender
from utils import *

In [ ]:
@dataclass
class Args:
    dataset: str
    data_root: str = "data/"
    train_dir: str = "/test/train.txt"
    test_dir: str = "/test/test.txt"
    L: int = 5
    T: int = 3
    n_iter: int = 40
    seed: int = 123
    batch_size: int = 512
    learning_rate: float = 1e-3
    l2: float = 5e-6
    neg_samples: int = 3
    use_cuda: bool = True
    save_root: str = "checkpoints/"
    model_type: str = "cnn"
    d: int = 50
    block_num: int = 2
    block_dim: List[int] = field(default_factory=lambda: [128, 256])
    drop: float = 0.5
    fc_dim: int = 150
    ac_fc: str = "tanh"


def load_instance(args: Args, saved_state_path: str = "") -> Tuple[Recommender, Interactions, Interactions, int, float]:
    # set seed
    set_seed(args.seed,
             cuda=args.use_cuda)
    # load dataset
    train = Interactions(args.data_root+args.dataset+args.train_dir)
    # transform triplets to sequence representation
    train.to_sequence(args.L, args.T)

    test = Interactions(args.data_root+args.dataset+args.test_dir,
                        user_map=train.user_map,
                        item_map=train.item_map)

    # Instantiate Model
    model = Recommender(args)

    # Initialize the Model
    model._initialize(train)

    best_map = -1
    epoch = 0

    if saved_state_path:
        saved_state = torch.load(saved_state_path, map_location=model._device)

        # If provided, load the pytorch state
        if "state_dict" in saved_state:
            model._net.load_state_dict(saved_state["state_dict"])
        if "optimizer" in saved_state:
            model._optimizer.load_state_dict(saved_state["optimizer"])
        if "best_map" in saved_state:
            best_map = saved_state["best_map"]
        if "epoch" in saved_state:
            epoch = saved_state["epoch"]

    return model, train, test, epoch, best_map


@dataclass
class EvaluationResult:
    map: float
    prec1: float
    prec5: float
    prec10: float
    recall1: float
    recall5: float
    recall10: float


def evaluate(model, test, train) -> EvaluationResult:
    precision, recall, mean_aps = evaluate_ranking(
        model, test, train, k=[1, 5, 10])

    return EvaluationResult(mean_aps, *[np.mean(p) for p in precision], *[np.mean(r) for r in recall])


def display_result(result: EvaluationResult) -> None:
    md = "| Metric | Value |\n| --- | --- |\n"
    md += f"| MAP | {result.map:.4f} |\n"
    md += f"| Precision (@1) | {result.prec1:.4f} |\n"
    md += f"| Precision (@5) | {result.prec5:.4f} |\n"
    md += f"| Precision (@10) | {result.prec10:.4f} |\n"
    md += f"| Recall (@1) | {result.recall1:.4f} |\n"
    md += f"| Recall (@5) | {result.recall5:.4f} |\n"
    md += f"| Recall (@10) | {result.recall10:.4f} |\n"
    display(Markdown(md))


## Create Model from Saved State

In [ ]:
model, train, test, epoch, best_map = load_instance(
    Args(
        dataset="ml1m",
        seed=123,
        use_cuda=True,
    ),
    saved_state_path="checkpoints/ml1m/ml1m_123.pth.tar"
)


## Evaluate Model

In [ ]:
evaluation_result = evaluate(model, train, test)
display_result(evaluation_result)

## Continue Training from Saved State

In [ ]:
# If the saved state did not include the best_map parameter, calculate it
if epoch > 0 and best_map < 0:
    best_map = evaluation_result.map

# Continue training the model
model.fit(train=train, test=test, verbose=True, start_epoch=epoch, best_map=best_map)